In [2]:
from alab_data import views
from alab_data import *
from alab_data.groups import Sample
from alab_data.utils.dev import drop_collections
from alab_data.units.nodes import WholeIngredient

In [3]:
drop_collections()

Add or pull all Actor's from the ActorView

In [4]:
actorview = views.ActorView()

In [5]:
actorview.get_by_tags(["SolidStateALab"])


[]

In [6]:
labman = Actor(
    name="LabMan 30-105", 
    tags=["SolidStateALab"],
    description="A robotic system that weighs and mixes powders in crucibles. Used to prepare samples for firing in furnaces. This is in building 30-105 at Lawrence Berkeley National Laboratory.")

tubefurnace1 = Actor(
    name="TubeFurnace1 30-105", 
    tags=["SolidStateALab"],
    description="A tube furnace in building 30-105 at Lawrence Berkeley National Laboratory.")

aeris = Actor(
    name="Aeris 30-105",
    tags=["SolidStateALab"],
    description="An x-ray diffractometer. This is in building 30-105 at Lawrence Berkeley National Laboratory.")

operator = Actor(
    name="Operator",
    description = "A generic actor for any operation performed by a human. No need to name names!")
actorview.add(labman)
actorview.add(tubefurnace1)
actorview.add(aeris)
actorview.add(operator)

ObjectId('6324ae46bbf4c9611e14a32e')

In [7]:
labman = actorview.get_by_name("LabMan 30-105")
labman

<Actor: LabMan 30-105>

Add or pull all analysis methods from AnalysisMethodView

In [8]:
analysismethodview = views.AnalysisMethodView()

In [9]:
cnn_phaseID = AnalysisMethod(
    name="CNN Phase ID", 
    tags=["SolidStateALab", "machinelearning"],
    description="A convolutional neural network trained to identify phases in x-ray diffraction patterns. doi: something/something")

analysismethodview.add(cnn_phaseID)

ObjectId('6324ae46bbf4c9611e14a330')

Add experimental data into the db

In [10]:
m0 = Material(
    name="Titanium Dioxide",
    formula="TiO2",
)

p0 = Action(    
    name="procurement",
    generated_materials=[m0],
    actor=operator,
)

In [11]:
p1 = Action(
    "grind",
    ingredients = [
        Ingredient(
            material=m0,
            amount=1,
            unit="g",
        )
    ],
    actor=operator,
    )
m1 = p1.generated_materials[0]

p2 = Action("sinter", ingredients = [WholeIngredient(m1)], actor=tubefurnace1)
m2 = p2.generated_materials[0]

p3 = Action("grind", ingredients = [WholeIngredient(m2)], actor=operator, final_step=True)
m3 = p3.generated_materials[0]

me0 = Measurement(
    name="XRD",
    material=m3,
    actor=aeris,
)

a0 = Analysis(
    name="Phase Identification",
    measurement=me0,
    analysis_method=cnn_phaseID
)

SampleView add to db

In [12]:
s = Sample(name="first_sample")
for n in [p0, m0, p1, m1, p2, m2, p3, m3, me0, a0]:
    s.add_node(n)

In [13]:
from alab_data.views.groups import SampleView
sv = SampleView()

In [14]:
s.to_dict()

{'_id': ObjectId('6324ae47bbf4c9611e14a33b'),
 'name': 'first_sample',
 'description': '',
 'nodes': {'Material': [ObjectId('6324ae47bbf4c9611e14a331'),
   ObjectId('6324ae47bbf4c9611e14a334'),
   ObjectId('6324ae47bbf4c9611e14a336'),
   ObjectId('6324ae47bbf4c9611e14a338')],
  'Action': [ObjectId('6324ae47bbf4c9611e14a332'),
   ObjectId('6324ae47bbf4c9611e14a333'),
   ObjectId('6324ae47bbf4c9611e14a335'),
   ObjectId('6324ae47bbf4c9611e14a337')],
  'Analysis': [ObjectId('6324ae47bbf4c9611e14a33a')],
  'Measurement': [ObjectId('6324ae47bbf4c9611e14a339')]},
 'tags': []}

In [15]:
sv.add(s)

ObjectId('6324ae47bbf4c9611e14a33b')